# Projet: apports de Spark à la recommandation et au clustering de vins

On considère un dataset dans lequel est proposé différents vins, une note, un prix ainsi qu'une description donnée par une personne ayant gouté ce vin. Le but de ce projet est de proposer deux chaines de traitement, une sans Spark et une avec, afin de réaliser les deux opérations suivantes:
- recommandations de vins basés sur la cosine similarité entre les descriptions des vin.
- clustering des vins en une dizaine de cluster afin de déterminer des groupes.

**Link to data**:
https://www.kaggle.com/zynicide/wine-reviews#winemag-data-130k-v2.csv

In [1]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import collections
import numpy as np
import time
import pandas as pd
import nltk
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csc_matrix
from sklearn.cluster import KMeans as skKMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import FeatureHasher

import pyspark
import findspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType
from pyspark.sql.functions import udf,col
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans as pysparkKMeans
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Word2Vec
from pyspark.sql.types import FloatType

PATH_TO_DATA = r"C:/Users/assae/Documents/Cours CS/3A/OMA/Big Data/spark_wine_project/"

## Préparation du dataframe

In [8]:
a = time.time()

df = pd.read_csv(PATH_TO_DATA + "winemag-data-130k-v2.csv", index_col=0) 
 
todrop = ['designation', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle']    
    
df = df.drop(todrop, axis=1)
df = df.dropna().reset_index(drop=True)
df = df.drop_duplicates(subset='title').reset_index(drop=True)
df[["points", "price"]] = df[["points", "price"]].apply(pd.to_numeric)
print(df.shape)

dict_wine_description = dict(zip(list(df['title']), list(df['description'])))

df.head()

b = time.time()
print("Time to extract data: ", b-a)

(110582, 8)
Time to extract data:  1.7497515678405762


# 1. Première méthode : sans Spark

## 1.1. Recommandation basée sur des vins ayant une description similaire

### 1.1.1. Nettoyage des descrptions

In [9]:
stop_words = list(stopwords.words('english'))

def stem(string):
    stemmer = SnowballStemmer("english")
    string = string.split(" ")
    chaine = ""
    for i in range(len(string)):
        chaine += " " + stemmer.stem(string[i])
    return chaine

def clean_txt(txt):
    global stop_words
    
    # lower text
    txt = txt.lower()
    ### special escaping character '...'
    txt = txt.replace(u'\u2026','.')
    txt = txt.replace(u'\u00a0',' ')
    ### remove non alphanumeric char
    txt = re.sub("[^a-zA-Z0-9 ]", '', txt)
    ### remove stop words
    txt1 = txt.split(" ")
    txt2 = [x for x in txt1 if x not in stop_words]
    txt0 = ""
    for i in range(len(txt2)):
        txt0 += " " + txt2[i]
    txt = txt0 
    ### stemming
    txt = stem(txt)
    return txt

In [10]:
# Test

title = df['title'][0]

print(dict_wine_description[title])
clean_txt(dict_wine_description[title])

This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.


'  ripe fruiti wine smooth still structur firm tannin fill juici red berri fruit freshen acid  alreadi drinkabl although certain better 2016'

In [11]:
def clean_descriptions(dico):
    a = time.time()
    cleaned = []
    keys = list(dico.keys())
    values = list(dico.values())
    for i in range(len(values)):
        cleaned.append(clean_txt(values[i]))
    b = time.time()
    print("Time to clean the descriptions : ", b-a)  
    return dict(zip(keys, cleaned))

In [12]:
dict_cleaned_descriptions = clean_descriptions(dict_wine_description)

Time to clean the descriptions :  78.15451908111572


### 1.1.2. Vectorisation des descriptions

In [13]:
def one_hot_encoding(dict_wines):
    a = time.time()
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(list(dict_wines.values()))
    b = time.time()
    print("Time to encode descriptions : ", b-a)
    return dict(zip(list(dict_wines.keys()), X)), X

def tf_idf(dict_wines):
    a = time.time()
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(list(dict_wines.values()))
    b = time.time()
    print("Time to encode descriptions : ", b-a)
    return dict(zip(list(dict_wines.keys()), X)), X

def hachage(dict_wines):
    a = time.time()
    values = list(dict_wines.values())
    tohash = []
    for sent in values:
        cnt = collections.Counter()
        for word in sent.split():
            cnt[word] += 1
        tohash.append(dict(cnt))
    h = FeatureHasher(n_features=10)
    X = h.transform(tohash)
    b = time.time()
    print("Time to encode descriptions : ", b-a)
    return dict(zip(list(dict_wines.keys()), X)), X

In [17]:
title = df['title'][0]
a,b = one_hot_encoding(dict_cleaned_descriptions)

Time to encode descriptions :  3.297089099884033


In [18]:
title = df['title'][0]
a,b = tf_idf(dict_cleaned_descriptions)

Time to encode descriptions :  3.142932415008545


In [21]:
title = df['title'][0]
a,b = hachage(dict_cleaned_descriptions)

Time to encode descriptions :  2.8509058952331543


### 1.1.3. Recommandations

In [28]:
def score(v1, v2):
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    return cos_sim


def recom_knn(wine, dict_wines, k, model):
    keys = list(dict_wines.keys())
    if model == 'one-hot':
        dict_encoding, encoding = one_hot_encoding(dict_wines)
    elif model == 'tfidf':
        dict_encoding, encoding = tf_idf(dict_wines)
    elif model == 'hash':
        dict_encoding, encoding = hachage(dict_wines)
    else:
        print("Model can be 'one-hot', 'tfidf', or 'hash' only")
        return None
    a = time.time()
    scores = []
    values = list(dict_encoding.values())
    for i in range(len(dict_encoding)):
        considered_wine = np.array(dict_encoding[wine].todense())[0]
        scores.append(score(considered_wine, np.array(values[i].todense())[0]))
    b = time.time()
    print("Time to compute cosinus similarities : ", b-a)
    sorted_index = np.argsort(-np.array(scores))
    wines = []
    for j in range(1, k+1):
        wines.append((keys[sorted_index[j]], scores[sorted_index[j]]))
    c = time.time()
    print("Time to sort the recommandations : ", c-b)
    return wines

In [29]:
title = df['title'][0]
print(title)
recom_knn(title, dict_cleaned_descriptions, 10, 'one-hot')

Quinta dos Avidagos 2011 Avidagos Red (Douro)
Time to encode descriptions :  4.081492900848389
Time to compute cosinus similarities :  38.19250464439392
Time to sort the recommandations :  0.03124403953552246


[('Enoport 2015 Vilalva Reserva Red (Douro)', 0.5821817364274594),
 ('Château de Pizay 2014  Régnié', 0.5455447255899809),
 ('Les Frères Perroud 2014 Vieilles Vignes  (Brouilly)', 0.5238095238095238),
 ('Château Bertinerie 2011 Grande Cuvée  (Blaye Côtes de Bordeaux)',
  0.5238095238095238),
 ('Domaine Chasselay 2010 Les Grands Eparcieux  (Beaujolais)',
  0.5238095238095238),
 ('Domaine les Pins 2012 Le Clos Cabernet Franc (Bourgueil)',
  0.5135525910130956),
 ('Schröder & Schÿler 2012 Private Réserve  (Médoc)', 0.5095101710852534),
 ('Manuel Olivier 2012  Pommard', 0.5039526306789697),
 ('Château Bélingard 2012 Réserve Red (Côtes de Bergerac)',
  0.5039526306789697),
 ('Casa Santos Lima 2012 Quinta dos Bons Ventos Red (Lisboa)',
  0.5006261743217588)]

In [30]:
title = df['title'][0]
print(title)
recom_knn(title, dict_cleaned_descriptions, 10, 'tfidf')

Quinta dos Avidagos 2011 Avidagos Red (Douro)
Time to encode descriptions :  3.5968830585479736
Time to compute cosinus similarities :  34.32163858413696
Time to sort the recommandations :  0.03124260902404785


[('Enoport 2015 Vilalva Reserva Red (Douro)', 0.49097115516236123),
 ('Joseph Drouhin 2012  Moulin-à-Vent', 0.42944709961586114),
 ('Casa Santos Lima 2012 Quinta dos Bons Ventos Red (Lisboa)',
  0.4246471882115379),
 ("Fabien Collonge 2014 L'Aurore des Côtes  (Chiroubles)", 0.4010955738952229),
 ('Les Frères Perroud 2014 Vieilles Vignes  (Brouilly)', 0.39931391451266035),
 ('Château Taussin 2014 Premium  (Bordeaux)', 0.39754375425327115),
 ("Emile Beyer 2013 L'Hostellerie Pinot Gris (Alsace)", 0.39257398556680123),
 ('Antonin Rodet 2011 Château de Mercey  (Santenay)', 0.39225414809727677),
 ('Château Coutinel 2011 Red (Fronton)', 0.3916813309252866),
 ('Cave de Saumur 2014 Réserve des Vignerons  (Saumur)', 0.383444784144407)]

In [31]:
title = df['title'][0]
print(title)
recom_knn(title, dict_cleaned_descriptions, 10, 'hash')

Quinta dos Avidagos 2011 Avidagos Red (Douro)
Time to encode descriptions :  3.2896358966827393


C:\Users\assae\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Time to compute cosinus similarities :  4.751907587051392
Time to sort the recommandations :  0.015624284744262695


[('CARM 2005 Reserva Red (Douro)', 0.9772545497599154),
 ('Château Jeanguillon 2014  Bordeaux Supérieur', 0.9713237285143653),
 ('Columbia Crest 2009 Two Vines Pinot Grigio (Washington)',
  0.9607689228305228),
 ('Manciat-Poncet 2014 Les Crays  (Pouilly-Fuissé)', 0.9607689228305228),
 ('Wellington 2004 Estate Merlot (Sonoma Valley)', 0.9502621934663978),
 ('Quinta do Crasto 2012 Flor de Crasto Red (Douro)', 0.949157995752499),
 ('Château de Gaudou 2010 Grand Lignée Malbec-Merlot (Cahors)',
  0.9467292624062573),
 ('Château de Santenay 2013  Mercurey', 0.9435641951204965),
 ('Steininger 2011 Rosé Sekt Cabernet Sauvignon (Österreichischer Sekt)',
  0.9435641951204965),
 ('Canard-Duchêne NV Charles VII Grande Cuvée de la Rosé Brut  (Champagne)',
  0.9433700705169153)]

## 1.2. Clustering des vins en prenant en compte l'ensemble des informations

In [32]:
def clustering_wines(data, clean_dict_wines, model='tfidf', nb_clusters=10):
    if model == 'one-hot':
        dict_encoding, encoding = one_hot_encoding(clean_dict_wines)
        a = time.time()
        pca_subs = TruncatedSVD(n_components=50)
        reduced_encoding = pd.DataFrame(pca_subs.fit_transform(encoding))
        b = time.time()
        pca_time = b-a
        print("Time PCA : ", pca_time)
    elif model == 'tfidf':
        a = time.time()
        dict_encoding, encoding = tf_idf(clean_dict_wines)
        pca_subs = TruncatedSVD(n_components=50)
        reduced_encoding = pd.DataFrame(pca_subs.fit_transform(encoding))
        b = time.time()
        pca_time = b-a
        print("Time PCA : ", pca_time)
    elif model == 'hash':
        dict_encoding, encoding = hachage(clean_dict_wines)
        a = time.time()
        reduced_encoding = pd.DataFrame(encoding.todense())
        b = time.time()
        pca_time = b-a
        print("Time to convert to dense vectors : ", pca_time)
    else:
        print("Model can be 'one-hot', 'tfidf', or 'hash' only")
        return None
    c = time.time()
    df_clustering = pd.concat([df["points"], df["price"], pd.get_dummies(df["country"]), pd.get_dummies(df["province"]), pd.get_dummies(df["variety"]), reduced_encoding], axis = 1)
    clustering = skKMeans(n_clusters=nb_clusters)
    clustered = clustering.fit_predict(df_clustering)
    df_final = pd.concat([df["title"], pd.DataFrame(clustered, columns=['cluster'])], axis=1)  
    d = time.time()
    print("Time for clustering: ", d-c+pca_time)
    return df_final        

In [33]:
df_final = clustering_wines(df, dict_cleaned_descriptions, model='one-hot', nb_clusters=10)
df_final['cluster'].value_counts()

Time to encode descriptions :  3.5217132568359375
Time PCA :  6.7115278244018555
Time for clustering:  148.17302799224854


5    48199
0    33716
3    18565
8     7360
7     1934
1      558
4      178
9       54
2       11
6        7
Name: cluster, dtype: int64

In [34]:
df_final = clustering_wines(df, dict_cleaned_descriptions, model='tfidf', nb_clusters=10)
df_final['cluster'].value_counts()

Time to encode descriptions :  3.999610424041748
Time PCA :  16.580242395401
Time for clustering:  147.1545774936676


0    44746
6    30730
5    19950
2     9251
7     3640
8     1642
1      420
4      162
9       34
3        7
Name: cluster, dtype: int64

In [35]:
df_final = clustering_wines(df, dict_cleaned_descriptions, model='hash', nb_clusters=10)
df_final['cluster'].value_counts()

Time to encode descriptions :  4.118942022323608
Time to convert to dense vectors :  0.006985187530517578
Time for clustering:  156.14875507354736


1    46536
9    31180
6    20922
0     8960
3     2175
7      559
5      178
2       55
8       10
4        7
Name: cluster, dtype: int64

# 2. Deuxième méthode : avec Spark

In [2]:
findspark.init("C:/spark/spark-2.3.2-bin-hadoop2.7")
sc=pyspark.SparkContext("local[*]","Project")
sqlc=SQLContext(sc) 

In [3]:
a = time.time()

sdf = sqlc.read.csv(PATH_TO_DATA + "winemag-data-130k-v2.csv", header=True)

todrop = ['designation', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle']

for c in todrop:
    sdf = sdf.drop(c)
    
sdf = sdf.na.drop()
sdf = sdf.dropDuplicates(['title'])
sdf.count()

b = time.time()

print("Time to extract data:", b-a)

Time to extract data: 13.705365419387817


## 2.1. Recommandation basée sur des vins ayant une description similaire

### 2.1.1. Traitement du texte

In [4]:
def stem(liste):
    stemmer = SnowballStemmer("english")
    l2 = []
    for i in range(len(liste)):
        l2.append(stemmer.stem(liste[i]))
    return l2

def clean_txt(df):
    a = time.time()
    regexTokenizer = RegexTokenizer(inputCol="description", outputCol="words", pattern="\\W")
    df = regexTokenizer.transform(df)
    df = df.drop('description')
    remover = StopWordsRemover(inputCol="words", outputCol="desc")
    df = remover.transform(df)
    df = df.drop('words')
    fct = udf(lambda x: stem(x), ArrayType(StringType()))
    df = df.withColumn("description", fct("desc"))
    df = df.drop('desc')
    b = time.time()
    print("Time to clean the descriptions : ", b-a)
    return df
    

In [5]:
sdfc = clean_txt(sdf)

Time to clean the descriptions :  0.5016565322875977


### 2.1.2. Vectorisation des descriptions

In [6]:
def tfidf(df):
    a = time.time()
    hashingTF = HashingTF(inputCol="description", outputCol="rawFeatures", numFeatures=10)
    df = hashingTF.transform(df)
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(df)
    df = idfModel.transform(df)
    df = df.drop('description')
    df = df.drop('rawFeatures')
    b = time.time()
    print("Time to encode the descriptions : ", b-a)
    return df

def word2vec(df):
    a = time.time()
    word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="description", outputCol="features")
    model = word2Vec.fit(df)
    df = model.transform(df)
    df = df.drop('description')
    b = time.time()
    print("Time to encode the descripions : ", b-a)
    return df
    

In [7]:
sdf_tfidf = tfidf(sdfc)

Time to encode the descriptions :  269.55376410484314


In [8]:
sdf_w2v = word2vec(sdfc)

Time to encode the descripions :  524.3509228229523


### 2.1.3. Recommandations

In [9]:
def score(v1, v2):
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    return cos_sim

def reco_spark(wine_name, sdf, k=10, model='tfidf'):
    a = time.time()
    try:
        sdf.createTempView(model)
        df = sqlc.sql("SELECT title, features FROM " + model).toPandas()
    except:
        df = sqlc.sql("SELECT title, features FROM " + model).toPandas()
    b = time.time()
    print("Time to select the data :", b-a)
    c  = time.time()
    l = zip(list(df['title']), list(df['features']))
    rdd = sc.parallelize(l)
    wine_encoding = list(df[df['title'] == wine_name].features)[0]
    rdd1 = rdd.map(lambda s: (s[0], score(s[1], wine_encoding)))
    d  = time.time()
    print("Time for similiraty calculations :", d-c)
    rdd2 = rdd1.sortBy(lambda x: -x[1])
    e = time.time()
    print("Time to sort results :", e-d)
    final = rdd2.take(k)
    f = time.time()
    print("Time to collect results :", f-e)
    return final

In [10]:
wine = 'Quinta dos Avidagos 2011 Avidagos Red (Douro)'
reco_spark(wine, sdf_tfidf, k=10, model='tfidf')

Time to select the data : 241.34242367744446
Time for similiraty calculations : 0.5399210453033447
Time to sort results : 64.20555782318115
Time to collect results : 35.38589310646057


[('Quinta dos Avidagos 2011 Avidagos Red (Douro)', 0.9999999999999999),
 ('Edmeades 2011 Zinfandel (Mendocino County)', 0.9643488391799788),
 ('Winzerkeller Andau 2013 St. Laurent (Burgenland)', 0.9622234035643586),
 ('Manuel Olivier 2011  Bourgogne', 0.9608204941230747),
 ('Casal do Conde 2012 Quinta da Arrancosa Moscatel Graúdo (Tejo)',
  0.9591004951560843),
 ('F X Pichler 2006 Von de Terrassen Federspiel Riesling (Wachau)',
  0.9590022053156014),
 ('Maryhill 2009 Cabernet Sauvignon (Columbia Valley (WA))',
  0.956376213374129),
 ('Apsara 2014 Kick Ranch Sauvignon Blanc (Sonoma County)',
  0.9527419069308315),
 ('Harlow Ridge 2011 Chardonnay (Lodi)', 0.9520437122175613),
 ('Quinta do Casal Branco 2015 Tinto Red (Tejo)', 0.9508761648408962)]

In [11]:
wine = 'Quinta dos Avidagos 2011 Avidagos Red (Douro)'
reco_spark(wine, sdf_w2v, k=10, model='w2v')

Time to select the data : 245.00079083442688
Time for similiraty calculations : 0.30518627166748047
Time to sort results : 8.79548954963684
Time to collect results : 5.612997770309448


[('Quinta dos Avidagos 2011 Avidagos Red (Douro)', 1.0000000000000002),
 ('Wines & Winemakers 2011 Companhia das Lezírias Catapereiro Red (Tejo)',
  0.9942232348354221),
 ('Château Lestage 2013  Listrac-Médoc', 0.9935354202240964),
 ('Château Jeanguillon 2014  Bordeaux Supérieur', 0.993463297087076),
 ("Poças 2013 Coroa d'Ouro Red (Douro)", 0.9934096013505216),
 ('Château le Grand Verdus 2015  Bordeaux Supérieur', 0.9925105526675456),
 ('Louis Max 2014 Domaine la Marche  (Mercurey)', 0.9920481591549125),
 ('Château du Perier 2011  Médoc', 0.9915061932799112),
 ('Château Cormeil-Figeac 2013  Saint-Émilion', 0.9912598259172178),
 ('Cortes de Cima 2013 Cabernet Sauvignon (Alentejano)', 0.9910822433519998)]

## 2.2. Clustering des vins

In [12]:
def encode_categorical_features(sdf):
    a = time.time()
    sdf = sdf.drop('winery')
    sdf = sdf.drop('title')
    sdf = sdf.drop('_c0')
    to_encode = ['country', 'province', 'variety']
    for cat in to_encode:
        stringIndexer = StringIndexer(inputCol=cat, outputCol=cat + "Index")
        model = stringIndexer.fit(sdf)
        indexed = model.transform(sdf)
        encoder = OneHotEncoder(inputCol=cat + "Index", outputCol=cat + "Vec")
        sdf = encoder.transform(indexed)
        sdf = sdf.drop(cat)
        sdf = sdf.drop(cat+'Index')
    sdf = sdf.withColumn("points", sdf["points"].cast(FloatType()))
    sdf = sdf.withColumn("price", sdf["price"].cast(FloatType()))
    sdf = sdf.na.drop()
    assembler = VectorAssembler(inputCols=["provinceVec", "countryVec", "varietyVec", "features", "points", "price"], outputCol="feat")
    output = assembler.transform(sdf)
    for cat in ["provinceVec", "countryVec", "varietyVec", "features", "points", "price"]:
        output = output.drop(cat)
    b = time.time()
    print("Time to prepare the dataset for clustering : ", b-a)
    return output

In [13]:
sdf_enc_tfidf = encode_categorical_features(sdf_tfidf)

Time to prepare the dataset for clustering :  24.029553174972534


In [14]:
sdf_enc_w2v = encode_categorical_features(sdf_w2v)

Time to prepare the dataset for clustering :  20.689386129379272


In [15]:
def clustering(sdf):
    a = time.time()
    kmeans = pysparkKMeans(featuresCol='feat', predictionCol='prediction', k=10, seed=1)
    model = kmeans.fit(sdf)
    predictions = model.transform(sdf)
    b = time.time()
    print("Time for clustering : ", b-a)
    return predictions

In [18]:
predictions_tfidf = clustering(sdf_enc_tfidf)
a = time.time()
predictions_tfidf.groupBy('prediction').count().show()
b = time.time()
print("Time to collect the results : ", b-a)

Time for clustering :  279.9073181152344
+----------+-----+
|prediction|count|
+----------+-----+
|         1|  409|
|         6|   69|
|         3|    3|
|         5| 2037|
|         9|    7|
|         4|13228|
|         8|19451|
|         7| 5996|
|         2|27491|
|         0|41866|
+----------+-----+

Time to collect the results :  248.48058104515076


In [19]:
predictions_w2v = clustering(sdf_enc_w2v)
a = time.time()
predictions_w2v.groupBy('prediction').count().show()
b = time.time()
print("Time to collect the results : ", b-a)

Time for clustering :  275.7115843296051
+----------+-----+
|prediction|count|
+----------+-----+
|         1|  409|
|         6|   69|
|         3|    3|
|         5| 2037|
|         9|    7|
|         4|13179|
|         8|19451|
|         7| 5996|
|         2|27107|
|         0|42299|
+----------+-----+

Time to collect the results :  246.67991542816162
